# SkinologyKE MobileNetV2 Training Notebook
This notebook trains a lightweight skin condition classifier using MobileNetV2. Datasets: HAM10000, SD-260, Fitzpatrick17k.

## 1. Install Dependencies
Install TensorFlow, pandas, scikit-learn, and other required libraries.

In [ ]:
!pip install tensorflow pandas scikit-learn matplotlib pillow

## 2. Load Datasets
Download and load HAM10000, SD-260, and Fitzpatrick17k datasets. (Use sample code for HAM10000; adapt for others.)

In [ ]:
import pandas as pd
import os
from glob import glob

# Example: Load HAM10000 metadata
ham_meta = pd.read_csv('HAM10000_metadata.csv')
image_dir = 'HAM10000_images/'
image_paths = glob(os.path.join(image_dir, '*.jpg'))

# Display sample
ham_meta.head()

## 2a. Load SD-260 Dataset
Download and load SD-260 metadata and images. Adjust paths as needed for your environment.

In [ ]:
# Example: Load SD-260 metadata
sd260_meta = pd.read_csv('SD-260_metadata.csv')
sd260_image_dir = 'SD-260_images/'
sd260_meta['img_path'] = sd260_meta['image_id'].apply(lambda x: os.path.join(sd260_image_dir, f'{x}.jpg'))
# Display sample
sd260_meta.head()

## 2b. Load Fitzpatrick17k Dataset
Download and load Fitzpatrick17k metadata and images. Adjust paths as needed for your environment.

In [ ]:
# Example: Load Fitzpatrick17k metadata
fitz_meta = pd.read_csv('Fitzpatrick17k_metadata.csv')
fitz_image_dir = 'Fitzpatrick17k_images/'
fitz_meta['img_path'] = fitz_meta['image_id'].apply(lambda x: os.path.join(fitz_image_dir, f'{x}.jpg'))
# Display sample
fitz_meta.head()

## 3. Preprocess Images and Labels
Resize images, normalize, and encode labels. Combine datasets if needed.

In [ ]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder

IMG_SIZE = 224

def load_and_preprocess(img_path):
    img = Image.open(img_path).resize((IMG_SIZE, IMG_SIZE))
    arr = np.array(img) / 255.0
    if arr.shape[-1] == 4:
        arr = arr[..., :3]
    return arr

# Example: preprocess HAM10000
ham_meta['img_path'] = ham_meta['image_id'].apply(lambda x: os.path.join(image_dir, f'{x}.jpg'))
X = np.stack([load_and_preprocess(p) for p in ham_meta['img_path']])
le = LabelEncoder()
y = le.fit_transform(ham_meta['dx'])

print('X shape:', X.shape)
print('y shape:', y.shape)

## 3a. Advanced Data Augmentation
Apply random flips, rotations, and color jitter to improve generalization.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
# Example: datagen.flow(X_train, y_train)

## 3b. Class Balancing and Multi-Dataset Merging
Combine all datasets and balance classes for robust training.

In [ ]:
# Merge all datasets
all_meta = pd.concat([ham_meta, sd260_meta, fitz_meta], ignore_index=True)
all_X = np.stack([load_and_preprocess(p) for p in all_meta['img_path']])
all_y = le.fit_transform(all_meta['dx'])

# Class balancing (undersample/oversample example)
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
all_X_rs, all_y_rs = ros.fit_resample(all_X.reshape((all_X.shape[0], -1)), all_y)
all_X_rs = all_X_rs.reshape((-1, IMG_SIZE, IMG_SIZE, 3))

print('Balanced class distribution:', Counter(all_y_rs))

## 3c. Save Label Mappings to labels.json
Export label encoder classes for downstream inference.

In [ ]:
import json
label_map = {str(i): c for i, c in enumerate(le.classes_)}
with open('labels.json', 'w') as f:
    json.dump(label_map, f)
print('Saved label mappings to labels.json')

## 4. Build and Compile MobileNetV2 Model
Use TensorFlow/Keras to build a transfer learning model.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
preds = Dense(len(le.classes_), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 5. Train and Evaluate Model
Train the model and plot accuracy/loss curves.

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## 5a. Evaluate Model on External Test Set
Test model performance on a held-out dataset (e.g., Fitzpatrick17k test split).

In [ ]:
# Example: External test evaluation
fitz_test = fitz_meta.sample(frac=0.2, random_state=42)
X_test = np.stack([load_and_preprocess(p) for p in fitz_test['img_path']])
y_test = le.transform(fitz_test['dx'])
results = model.evaluate(X_test, y_test)
print('External test loss, accuracy:', results)

## 6. Export Trained Model
Save the trained model as .h5 and TFLite formats for deployment.

In [ ]:
# Save as .h5
model.save('mobilenetv2_skinologyke.h5')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('mobilenetv2_skinologyke.tflite', 'wb') as f:
    f.write(tflite_model)

## 6a. Upload Model to Hugging Face or Google Drive
Share your trained model for deployment or collaboration.

In [ ]:
# Upload to Google Drive (requires Google Colab)
from google.colab import drive

drive.mount('/content/drive')
!cp mobilenetv2_skinologyke.h5 /content/drive/MyDrive/
!cp mobilenetv2_skinologyke.tflite /content/drive/MyDrive/

# Upload to Hugging Face (requires huggingface_hub)
# !pip install huggingface_hub
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="mobilenetv2_skinologyke.h5",
    path_in_repo="mobilenetv2_skinologyke.h5",
    repo_id="your-username/skinologyke-model",
    repo_type="model"
)

## 7. Next Steps
- Test model on new images
- Integrate with Flask API and Streamlit frontend
- Document results and limitations

# Production Integration & Expansion Roadmap
This section outlines the steps to move each module from prototype to production, with actionable code and links to relevant files.

## 1. Before/After Gallery
- Integrate uploads with Firebase Storage/Firestore (see `firebase_config.py`).
- Build moderation dashboard and enforce consent.
- Enable multi-user gallery viewing.

In [ ]:
# Example: Firebase Storage upload (Python)
import firebase_admin
from firebase_admin import credentials, storage, firestore
cred = credentials.Certificate('path/to/serviceAccountKey.json')
firebase_admin.initialize_app(cred, {'storageBucket': 'your-bucket.appspot.com'})

def upload_image_to_firebase(image_path, user_id):
    bucket = storage.bucket()
    blob = bucket.blob(f'gallery/{user_id}/{os.path.basename(image_path)}')
    blob.upload_from_filename(image_path)
    # Save metadata to Firestore
    db = firestore.client()
    db.collection('gallery').add({'user_id': user_id, 'image_url': blob.public_url, 'approved': False})

## 2. Shop Payments
- Implement backend order tracking and inventory (see `api/endpoints/shop.py`).
- Integrate MPESA API for payments.
- Add order confirmation and receipts.

In [ ]:
# Example: MPESA payment integration (pseudo-code)
import requests
mpesa_url = 'https://api.safaricom.co.ke/mpesa/stkpush/v1/processrequest'
# ...prepare payload and headers...
response = requests.post(mpesa_url, json=payload, headers=headers)
if response.ok:
    print('Payment initiated')

## 3. Diagnosis API Integration
- Deploy Flask API to Render/Fly.io with HTTPS (see `api/app.py`).
- Add authentication and robust error handling.
- Implement user feedback loop.

In [ ]:
# Example: Deploy Flask API (Render)
# 1. Add requirements.txt and render.yaml
# 2. Push to GitHub and connect Render
# 3. Set up HTTPS and environment variables
# See deploy.md for details

## 4. Reminders & Routine Checklist
- Integrate Formspree/Google Calendar for live reminders.
- Sync routine checklist to backend for all users.
- Complete OneSignal/browser notification integration.

In [ ]:
# Example: Google Calendar API event creation (pseudo-code)
from googleapiclient.discovery import build
service = build('calendar', 'v3', credentials=creds)
event = {
    'summary': 'Skin Care Routine',
    'start': {'dateTime': '2025-07-23T09:00:00', 'timeZone': 'Africa/Nairobi'},
    'end': {'dateTime': '2025-07-23T09:30:00', 'timeZone': 'Africa/Nairobi'},
}
service.events().insert(calendarId='primary', body=event).execute()

## 5. Authentication
- Implement user profile, regimen/history saving (see `api/endpoints/auth.py`).
- Integrate login with other modules for seamless experience.

In [ ]:
# Example: Save diagnosis result to user profile (pseudo-code)
def save_diagnosis_to_profile(user_id, diagnosis):
    db = firestore.client()
    db.collection('users').document(user_id).update({'diagnosis_history': firestore.ArrayUnion([diagnosis])})

## 6. Blog
- Deploy markdown posts to GitHub Pages/Notion/Ghost (see `blog_content/`).
- Add SEO, images, and automate weekly posts.

In [ ]:
# Example: Publish markdown to GitHub Pages (pseudo-code)
# 1. Push markdown files to a GitHub repo
# 2. Enable Pages in repo settings
# 3. Use Jekyll or static site generator

## 7. Deployment & Hosting
- Complete deployment to Render, Streamlit Cloud, Firebase Hosting.
- Set up live URLs and SSL for API, frontend, and blog.

In [ ]:
# Example: Streamlit Cloud deployment
# 1. Add requirements.txt
# 2. Push to GitHub
# 3. Deploy via Streamlit Cloud dashboard

## 8. Expansion Features
- Test and complete PWA/offline experience (see `public/manifest.json`, `public/service-worker.js`).
- Implement skin tone detection, video consults, affiliate offers.

In [ ]:
# Example: Skin tone detection (pseudo-code)
def detect_skin_tone(image_path):
    img = Image.open(image_path).resize((100, 100))
    avg_color = np.mean(np.array(img), axis=(0, 1))
    # ...map avg_color to skin tone category...
    return avg_color

---
For full implementation, see referenced files and update each module as described. Use this roadmap to track progress and add new cells/code as features are completed.

# Implementation Roadmap: Missing Features
This section provides step-by-step code and instructions to implement all remaining MVP features for SkinologyKE.

## 1. AI Diagnostic Engine: Production API Deployment
- Deploy Flask API to Render/Fly.io:
    1. Add `requirements.txt` and `render.yaml` to `/api`.
    2. Push to GitHub.
    3. Create a new web service on Render/Fly.io, set environment variables, enable HTTPS.
    4. Update `/frontend/js/diagnose.js` to call the live API endpoint.

In [ ]:
# Upload trained model to Hugging Face Hub
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="mobilenetv2_skinologyke.h5",
    path_in_repo="mobilenetv2_skinologyke.h5",
    repo_id="your-username/skinologyke-model",
    repo_type="model"
)
print("Model uploaded to Hugging Face.")

## AI Diagnostic Engine: Frontend Integration
- In `/frontend/diagnose.html` and `/frontend/js/diagnose.js`, connect the upload form to the Flask API endpoint and display diagnosis results.
- In `/streamlit_app/main.py`, update to call the deployed API for inference.

## AI Diagnostic Engine: Referral Workflow
- Add WhatsApp deep link: `<a href='https://wa.me/254700000000?text=I+need+a+dermatologist+referral+for+my+SkinologyKE+diagnosis'>Contact Dermatologist</a>`
- Add Google Forms link: `<a href='https://forms.gle/your-google-form-id'>Referral Form</a>`

## 2. Dermatology Blog: Public Deployment & Automation
- Publish markdown posts to GitHub Pages:
    1. Push `/blog_content/*.md` to a GitHub repo.
    2. Enable Pages in repo settings.
    3. Use Jekyll for static site generation.
- Automate SEO meta tags and image formatting in blog markdown files.
- Set up GitHub Actions for weekly post automation.

In [ ]:
# Example: Add SEO meta tags to markdown
for md_file in ["acne-tips.md", "skin-lightening-risks.md", "eczema-vs-fungal.md", "pigmentation-facts.md"]:
    with open(f"../blog_content/{md_file}", "r+") as f:
        content = f.read()
        seo = f"<!-- SEO: Published 2025-07-22 -->\n"
        f.seek(0, 0)
        f.write(seo + content)
print("SEO meta tags added.")

## 3. Skin Routine Checklist & Reminder Tool
- Integrate Google Calendar API and Formspree/Zapier for reminders.
- Set up OneSignal/browser push notifications in `/frontend/js/reminders.js`.
- Ensure backend sync of routine data for all users via Firestore.

In [ ]:
# Example: Google Calendar event creation
from googleapiclient.discovery import build
service = build('calendar', 'v3', credentials=creds)
event = {
    'summary': 'Skin Care Routine',
    'start': {'dateTime': '2025-07-23T09:00:00', 'timeZone': 'Africa/Nairobi'},
    'end': {'dateTime': '2025-07-23T09:30:00', 'timeZone': 'Africa/Nairobi'},
}
service.events().insert(calendarId='primary', body=event).execute()
print("Routine reminder added to Google Calendar.")

## 4. Shop Module: Backend, Payments, Order Tracking
- Add product database and inventory management in Firestore (`/api/endpoints/shop.py`).
- Integrate Flutterwave/Paystack/MPESA payment API in backend and frontend.
- Implement order tracking and receipts in backend and `/frontend/shop.html`.

In [ ]:
# Example: Add product to Firestore
import firebase_admin
from firebase_admin import credentials, firestore
cred = credentials.Certificate('path/to/serviceAccountKey.json')
firebase_admin.initialize_app(cred)
db = firestore.client()
db.collection('products').add({
    'name': 'Cerave Cleanser',
    'price': 1200,
    'stock': 10,
    'skin_type': 'Oily',
    'ingredients': ['Ceramides', 'Niacinamide']
})
print("Product added to Firestore.")

## 5. Referral System: WhatsApp & Google Forms
- Add WhatsApp deep link and Google Forms to `/frontend/diagnose.html` and `/streamlit_app/main.py`.
- Set up dermatologist panel and email routing in backend (`/api/endpoints/referral.py`).

In [ ]:
# Example: Send referral email (pseudo-code)
def send_referral_email(user_email, diagnosis):
    # Use SMTP or email API
    # ...existing code...
    print(f"Referral sent for {user_email} with diagnosis: {diagnosis}")

## 6. Before/After Gallery: Firebase Integration & Moderation
- Connect gallery uploads to Firebase Storage/Firestore in `/frontend/js/firebase_init.js` and `/api/firebase_config.py`.
- Build admin dashboard for moderation in `/frontend/gallery.html`.
- Enable multi-user gallery viewing in frontend.

In [ ]:
# Example: Approve gallery image in Firestore
image_id = 'abc123'
db.collection('gallery').document(image_id).update({'approved': True})
print("Gallery image approved.")

## 7. Platform Architecture & Deployment
- Set up live URLs for API, frontend, and blog in `deploy.md`.
- Configure custom domain and SSL in Firebase/Render settings.
- Test PWA/offline experience using `public/manifest.json` and `public/service-worker.js`.

## 8. Expansion Options: Skin Tone, Video Consults, Affiliate Offers
- Implement skin tone detection in `/ai_model/export_model.py` and frontend.
- Add video consults via Jitsi/Zoom in `/frontend/pages/Consult.tsx`.
- Add affiliate offers in `/frontend/shop.html` and backend.

In [ ]:
# Example: Skin tone detection
from PIL import Image
import numpy as np
def detect_skin_tone(image_path):
    img = Image.open(image_path).resize((100, 100))
    avg_color = np.mean(np.array(img), axis=(0, 1))
    # Map avg_color to skin tone category
    return avg_color
print("Skin tone detected.")